# Python 实战项目

本教程包含一些综合性的实战项目，将前面学到的知识整合起来。

## 1. 学生管理系统

一个简单的学生信息管理系统，包含增删改查功能。


In [ ]:
# 学生管理系统（基础版）
class Student:
    """学生类"""
    
    def __init__(self, student_id, name, age, score):
        self.student_id = student_id
        self.name = name
        self.age = age
        self.score = score
    
    def __str__(self):
        return f"学号: {self.student_id}, 姓名: {self.name}, 年龄: {self.age}, 成绩: {self.score}"

class StudentManager:
    """学生管理器"""
    
    def __init__(self):
        self.students = {}
    
    def add_student(self, student):
        """添加学生"""
        if student.student_id in self.students:
            print(f"学号 {student.student_id} 已存在")
            return False
        self.students[student.student_id] = student
        print(f"添加成功: {student.name}")
        return True
    
    def remove_student(self, student_id):
        """删除学生"""
        if student_id in self.students:
            name = self.students[student_id].name
            del self.students[student_id]
            print(f"删除成功: {name}")
            return True
        print(f"学号 {student_id} 不存在")
        return False
    
    def find_student(self, student_id):
        """查找学生"""
        return self.students.get(student_id)
    
    def list_all_students(self):
        """列出所有学生"""
        if not self.students:
            print("暂无学生信息")
            return
        print("所有学生信息:")
        for student in self.students.values():
            print(f"  {student}")
    
    def get_statistics(self):
        """统计信息"""
        if not self.students:
            return None
        scores = [s.score for s in self.students.values()]
        return {
            'count': len(self.students),
            'average': sum(scores) / len(scores),
            'max': max(scores),
            'min': min(scores)
        }

# 使用示例
manager = StudentManager()

# 添加学生
manager.add_student(Student(1, "张三", 20, 85))
manager.add_student(Student(2, "李四", 21, 90))
manager.add_student(Student(3, "王五", 20, 78))

# 列出所有学生
manager.list_all_students()

# 查找学生
student = manager.find_student(2)
if student:
    print(f"\n找到学生: {student}")

# 统计信息
stats = manager.get_statistics()
if stats:
    print(f"\n统计信息:")
    print(f"  学生总数: {stats['count']}")
    print(f"  平均分: {stats['average']:.2f}")
    print(f"  最高分: {stats['max']}")
    print(f"  最低分: {stats['min']}")

# 删除学生
manager.remove_student(1)
print("\n删除后的学生列表:")
manager.list_all_students()


## 2. 待办事项管理器

一个简单的待办事项管理系统，使用文件持久化存储。


In [ ]:
# 待办事项管理器
import json
from datetime import datetime
from typing import List, Dict

class TodoItem:
    """待办事项类"""
    
    def __init__(self, title: str, description: str = "", priority: int = 1):
        self.id = id(self)  # 简单的ID生成
        self.title = title
        self.description = description
        self.priority = priority  # 1-低, 2-中, 3-高
        self.completed = False
        self.created_at = datetime.now().isoformat()
    
    def to_dict(self):
        """转换为字典"""
        return {
            'id': self.id,
            'title': self.title,
            'description': self.description,
            'priority': self.priority,
            'completed': self.completed,
            'created_at': self.created_at
        }
    
    @classmethod
    def from_dict(cls, data: Dict):
        """从字典创建对象"""
        item = cls(data['title'], data.get('description', ''), data.get('priority', 1))
        item.id = data['id']
        item.completed = data.get('completed', False)
        item.created_at = data.get('created_at', datetime.now().isoformat())
        return item
    
    def __str__(self):
        status = "✓" if self.completed else "○"
        priority_map = {1: "低", 2: "中", 3: "高"}
        return f"{status} [{priority_map[self.priority]}] {self.title}"

class TodoManager:
    """待办事项管理器"""
    
    def __init__(self, filename: str = "todos.json"):
        self.filename = filename
        self.todos: List[TodoItem] = []
        self.load()
    
    def add_todo(self, title: str, description: str = "", priority: int = 1) -> bool:
        """添加待办事项"""
        todo = TodoItem(title, description, priority)
        self.todos.append(todo)
        self.save()
        print(f"添加成功: {todo}")
        return True
    
    def complete_todo(self, todo_id: int) -> bool:
        """完成待办事项"""
        for todo in self.todos:
            if todo.id == todo_id:
                todo.completed = True
                self.save()
                print(f"完成: {todo.title}")
                return True
        print(f"ID {todo_id} 不存在")
        return False
    
    def list_todos(self, show_completed: bool = False):
        """列出待办事项"""
        filtered = self.todos if show_completed else [t for t in self.todos if not t.completed]
        if not filtered:
            print("暂无待办事项")
            return
        print("待办事项列表:")
        for i, todo in enumerate(filtered, 1):
            print(f"  {i}. {todo}")
    
    def save(self):
        """保存到文件"""
        data = [todo.to_dict() for todo in self.todos]
        try:
            with open(self.filename, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=2)
        except Exception as e:
            print(f"保存失败: {e}")
    
    def load(self):
        """从文件加载"""
        try:
            with open(self.filename, 'r', encoding='utf-8') as f:
                data = json.load(f)
                self.todos = [TodoItem.from_dict(item) for item in data]
        except FileNotFoundError:
            print("文件不存在，创建新的待办列表")
        except Exception as e:
            print(f"加载失败: {e}")

# 使用示例
todo_manager = TodoManager()

# 添加待办事项
todo_manager.add_todo("学习Python", "完成函数和类的学习", priority=3)
todo_manager.add_todo("买菜", "去超市买蔬菜和水果", priority=2)
todo_manager.add_todo("运动", "跑步30分钟", priority=1)

# 列出待办事项
todo_manager.list_todos()

# 完成一个待办
if todo_manager.todos:
    todo_manager.complete_todo(todo_manager.todos[0].id)

# 再次列出
print("\n完成一个事项后:")
todo_manager.list_todos()


## 3. 数据分析和可视化项目

使用Pandas和Matplotlib进行数据分析和可视化。


In [ ]:
# 数据分析项目示例
try:
    import pandas as pd
    import matplotlib.pyplot as plt
    import numpy as np
    
    # 设置中文字体
    plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei']
    plt.rcParams['axes.unicode_minus'] = False
    
    # 创建示例数据（销售数据）
    dates = pd.date_range('2024-01-01', periods=30, freq='D')
    sales_data = {
        '日期': dates,
        '销售额': np.random.randint(1000, 5000, 30),
        '产品A': np.random.randint(200, 800, 30),
        '产品B': np.random.randint(300, 900, 30),
        '产品C': np.random.randint(100, 600, 30)
    }
    
    df = pd.DataFrame(sales_data)
    df.set_index('日期', inplace=True)
    
    print("销售数据示例:")
    print(df.head())
    
    # 数据统计
    print("\n数据统计:")
    print(df.describe())
    
    # 计算总销售额
    total_sales = df['销售额'].sum()
    print(f"\n总销售额: {total_sales:,.2f}")
    
    # 平均每日销售额
    avg_daily = df['销售额'].mean()
    print(f"平均每日销售额: {avg_daily:,.2f}")
    
    # 产品销售额占比
    product_totals = df[['产品A', '产品B', '产品C']].sum()
    print("\n产品销售额:")
    for product, total in product_totals.items():
        percentage = (total / product_totals.sum()) * 100
        print(f"  {product}: {total:,} ({percentage:.1f}%)")
    
    print("\n数据可视化已准备就绪")
    print("可以使用以下代码绘制图表:")
    print("""
    # 绘制销售额趋势
    plt.figure(figsize=(10, 5))
    plt.plot(df.index, df['销售额'])
    plt.title('销售额趋势')
    plt.xlabel('日期')
    plt.ylabel('销售额')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    """)
    
except ImportError:
    print("需要安装: pip install pandas matplotlib numpy")


## 4. 增强版学生管理系统 (持久化 & 类型提示)

改进之前的学生管理系统，增加 JSON 文件持久化存储，并添加 Python 类型提示。


In [ ]:
import json
from typing import Dict, List, Optional
from dataclasses import dataclass, asdict

@dataclass
class Student:
    student_id: int
    name: str
    age: int
    score: float

class StudentManagerEnhanced:
    def __init__(self, filename: str = "students.json"):
        self.filename = filename
        self.students: Dict[int, Student] = {}
        self.load_data()

    def load_data(self) -> None:
        try:
            with open(self.filename, 'r', encoding='utf-8') as f:
                data = json.load(f)
                for item in data:
                    student = Student(**item)
                    self.students[student.student_id] = student
            print(f"成功加载 {len(self.students)} 名学生数据")
        except FileNotFoundError:
            print("数据文件不存在，初始化为空")
        except Exception as e:
            print(f"加载数据出错: {e}")

    def save_data(self) -> None:
        try:
            data = [asdict(s) for s in self.students.values()]
            with open(self.filename, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=2)
            print("数据保存成功")
        except Exception as e:
            print(f"保存数据出错: {e}")

    def add_student(self, student: Student) -> bool:
        if student.student_id in self.students:
            print(f"学号 {student.student_id} 已存在")
            return False
        self.students[student.student_id] = student
        self.save_data()
        return True

    def get_student(self, student_id: int) -> Optional[Student]:
        return self.students.get(student_id)

# 测试
manager = StudentManagerEnhanced()
manager.add_student(Student(101, "Alice", 20, 95.5))
manager.add_student(Student(102, "Bob", 21, 88.0))


## 5. 迷你项目：日志分析器

编写一个工具，分析模拟的服务器日志文件，统计访问量最高的 IP 和最常见的错误。


In [ ]:
import re
from collections import Counter
from datetime import datetime

# 模拟生成日志文件
log_content = """
2023-11-23 10:00:01 INFO 192.168.1.1 GET /index.html 200
2023-11-23 10:00:02 ERROR 192.168.1.2 GET /admin 403
2023-11-23 10:00:03 INFO 192.168.1.1 GET /about 200
2023-11-23 10:00:04 WARN 192.168.1.3 POST /login 401
2023-11-23 10:00:05 ERROR 192.168.1.2 GET /private 403
2023-11-23 10:00:06 INFO 192.168.1.4 GET /index.html 200
"""

with open("server.log", "w", encoding="utf-8") as f:
    f.write(log_content.strip())

def analyze_log(filename: str):
    ip_pattern = r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}"
    status_pattern = r"\s(\d{3})$"
    
    ip_counter = Counter()
    error_counter = Counter()
    
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line: continue
            
            # 提取IP
            ip_match = re.search(ip_pattern, line)
            if ip_match:
                ip_counter[ip_match.group()]+=1
            
            # 统计错误 (状态码 >= 400)
            status_match = re.search(status_pattern, line)
            if status_match:
                status = int(status_match.group(1))
                if status >= 400:
                    error_counter[status] += 1
    
    print("=== 分析结果 ===")
    print("访问最多的 IP:")
    for ip, count in ip_counter.most_common(3):
        print(f"  {ip}: {count} 次")
        
    print("\n错误状态码统计:")
    for status, count in error_counter.items():
        print(f"  {status}: {count} 次")

analyze_log("server.log")
